In [100]:
#Dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import time
import pandas as pd
import requests
from pprint import pprint
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [101]:
# URL of page to be scraped
url = 'https://en.wikipedia.org/wiki/List_of_radio_stations_in_New_York'

In [102]:
#Read In HTML
tables = pd.read_html(url)
#Create a Dateframe From First Table
table_df = pd.DataFrame(tables[0])
#Set Column Names From Index 0
table_df.columns = table_df.iloc[0]
#Drop Duplicate Row[0]
table_df = table_df.reindex(table_df.index.drop(0))
#Rename Weird Columns
table_df = table_df.rename(columns={"City of License [1][2]":"Location","Format [3]":"Format"})
#Sample Date
table_df.head()

,Call sign,Frequency,Location,Licensee,Format
1,WAAL,99.1 FM,Binghamton,"Townsquare Media Binghamton License, LLC",Classic rock
2,WABC,770 AM,New York City,Radio License Holdings LLC,News/Talk
3,WABH,1380 AM,Bath,"Tower Broadcasting, LLC",Sports
4,WABY,900 AM,Saratoga Springs,Empire Broadcasting Corporation,Silent
5,WACK,1420 AM,Newark,"Waynco Radio, Inc.",News/Talk Information


In [103]:
county_url = "https://en.wikipedia.org/wiki/List_of_towns_in_New_York_(state)"
cities_url = "https://en.wikipedia.org/wiki/List_of_cities_in_New_York_(state)"

In [104]:
county_table = pd.read_html(county_url)
cities_table = pd.read_html(cities_url)

In [105]:
county_df = pd.DataFrame(county_table[1])
cities_df = pd.DataFrame(cities_table[1])

In [106]:
#Set Column Names From Index 0
county_df.columns = county_df.iloc[0]
cities_df.columns = cities_df.iloc[0]
#Drop Duplicate Row[0]
county_df = county_df.reindex(county_df.index.drop(0))
cities_df = cities_df.reindex(cities_df.index.drop(0))


In [107]:
#Select Useful Columns
county_df = county_df[['Town','County']]
cities_df = cities_df[['City','County']]
#rename columns
county_df = county_df.rename(columns={"Town":"Location"})
cities_df = cities_df.rename(columns={"City":"Location"})


In [116]:
#append cities to towns 
final_counties_df = county_df.append(cities_df)

#data cleaning "New York" to "New York City"
final_counties_df["Location"] = final_counties_df["Location"].replace(
    {"New York":"New York City"})
#data cleaning "New York" to "New York City"
final_counties_df["County"] = final_counties_df["County"].replace(
    {"Bronx, Kings, New York, Queens, and Richmond":"Bronx"})

#Merge two data-frames
final_scrape_df = pd.merge(final_counties_df, table_df, on='Location')

#import radio csv as a dataframe 
radio_df = pd.read_csv("formats.csv")

#strip out extra spaces in the format values in final_scrape_df 
final_scrape_df.columns = final_scrape_df.columns.str.replace(' ','')


#join radio dataframe on "Format"
final_scrape_df = pd.merge(final_scrape_df,radio_df, on="Format")

#drop "Format"
final_scrape_df= final_scrape_df.drop(['Format'], axis=1)

#change Revised Format to Format 
final_scrape_df = final_scrape_df.rename(columns={"Revised Format":"Format"})

final_scrape_df




,Location,County,Callsign,Frequency,Licensee,Format
0,Alfred,Allegany,WALF,89.7 FM,Alfred University,Variety
1,Alfred,Allegany,WETD,90.7 FM,State University of New York,Variety
2,Angelica,Allegany,WRAQ-LP,92.7 FM,Angelica Community Radio Inc,Variety
3,Binghamton,Broome,WBDY-LP,99.5 FM,The Bundy Museum of History & Art,Variety
4,Binghamton,Broome,WBDY-LP,99.5 FM,The Bundy Museum of History & Art,Variety
5,Cazenovia,Madison,WITC,88.9 FM,Cazenovia College,Variety
6,Clinton,Clinton,WHCL-FM,88.7 FM,Trustees of Hamilton College,Variety
7,Clinton,Dutchess,WHCL-FM,88.7 FM,Trustees of Hamilton College,Variety
8,Corning,Steuben,WCEB,91.9 FM,Corning Community College,Variety
9,Corning,Steuben,WCEB,91.9 FM,Corning Community College,Variety


In [117]:
#Ping the endpoint thing
population_url = "https://api.census.gov/data/2010/dec/sf1?get=NAME,H010001&for=county:*&in=state:36&key=59a41ecf0dff091bd991b35c87176ffd56b8107d"


In [118]:
# Request data
population = requests.get(population_url).json()

#Create a dataframe
population_df = pd.DataFrame(population)

#create column headers
population_df.columns = population_df.iloc[0]
#remove old row[0]
population_df = population_df.reindex(population_df.index.drop(0))
#Split County, State Column
population_df["NAME"] = population_df["NAME"].str.split(" ", n = 1, expand = True)
#Rename Column as County
population_df = population_df.rename(columns={"NAME":"County","H010001":"Population"})
#Drop Unnecessary Columns
population_df = population_df[['County','Population']]


In [119]:
#Merge county graph with population with final_scrape which has radio data 
pop_radio_df = pd.merge(population_df, final_scrape_df, on ='County')
pop_radio_df.head()

,County,Population,Location,Callsign,Frequency,Licensee,Format
0,Genesee,58171,Batavia,WBTA,1490 AM,"HPL Communications, Inc.",Contemporary
1,Genesee,58171,Batavia,WBTA,1490 AM,"HPL Communications, Inc.",Contemporary
2,Genesee,58171,Batavia,WGCC-FM,90.7 FM,Genesee Community College,Rock
3,Genesee,58171,Batavia,WGCC-FM,90.7 FM,Genesee Community College,Rock
4,Greene,45821,Windham,WRIP,97.9 FM,"RIP Radio, LLC",Contemporary


In [120]:
#Determine # of formats per county
format_ser= pop_radio_df.groupby(['County', 'Format']).Format.count()
format_df=format_ser.unstack()
format_df_clean= format_df.fillna(0)
format_df_clean.head(50)

Format,Alternative,Classical,Contemporary,Country,Ethnic,Jazz,Oldies,Other,Religious,Rock,Spanish,Sports,Standards,Talk,Top40,Urban,Variety
County,,,,,,,,,,,,,,,,,
Albany,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0
Allegany,0.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
Bronx,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,2.0,2.0,0.0,3.0,0.0,4.0,1.0,2.0,0.0
Broome,0.0,0.0,1.0,4.0,0.0,0.0,0.0,2.0,3.0,3.0,0.0,2.0,0.0,6.0,0.0,0.0,2.0
Cattaraugus,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,7.0,4.0,0.0,1.0
Cayuga,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
Chautauqua,0.0,1.0,2.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,1.0
Chemung,0.0,0.0,1.0,2.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,0.0,3.0,4.0,0.0,0.0
Chenango,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0


In [125]:
#Add population
final_df = pd.merge(population_df, format_df_clean, on ='County')
final_df["County"] = final_df["County"].replace(
    {"Bronx":"New York City"})
final_df.head(52)

,County,Population,Alternative,Classical,Contemporary,Country,Ethnic,Jazz,Oldies,Other,Religious,Rock,Spanish,Sports,Standards,Talk,Top40,Urban,Variety
0,Genesee,58171,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Greene,45821,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Livingston,59635,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,Herkimer,63093,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0
4,Hamilton,4756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,Jefferson,110059,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,10.0,3.0,0.0,0.0
6,Lewis,26774,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,Niagara,212150,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0
8,Madison,68309,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
9,Monroe,718061,1.0,0.0,3.0,1.0,0.0,2.0,0.0,1.0,3.0,2.0,2.0,2.0,0.0,4.0,2.0,1.0,3.0


In [126]:
#cleanup since I'm getting duplicate column name error in sqlite
headers = list(final_df.columns.values)
headers

['County',
 'Population',
 'Alternative',
 'Classical',
 'Contemporary',
 'Country',
 'Ethnic',
 'Jazz',
 'Oldies',
 'Other',
 'Religious',
 'Rock',
 'Spanish',
 'Sports',
 'Standards',
 'Talk',
 'Top40',
 'Urban',
 'Variety']

In [134]:
#push to SQL.

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func


engine = create_engine('sqlite:///ny_radio.sqlite')
Base.metadata.create_all(engine)

final_df.to_sql(
    name= "NYStateofMind_db",
    con = engine,
    index = False,
    if_exists = 'replace'
)